In [2]:
import numpy as np
from os import listdir
import os
from os.path import isfile, join
from pathlib import Path


folder_path = "dataset/topo/embeddings/dnabert"


# list of files in the folder without the extension
IDs = [Path(f).stem for f in listdir(folder_path) if isfile(join(folder_path, f))]


embeddings_matrix = []


for id in IDs:

    # load the file as a numpy array
    raw_embedding = None
    try:
        raw_embedding = np.load(folder_path + "/" + id + ".npy")
        print(raw_embedding.shape)
    except:
        raw_embedding = np.load(folder_path + "/" + id + ".npy", allow_pickle=True)
        print(f"Error while loading the embedding of sequence {id}")
        # os.remove(folder_path + "/" + id + ".npy")
        print(raw_embedding.shape)
        

(3, 445, 768)
(1, 416, 768)
(2, 264, 768)
(2, 254, 768)
(1, 362, 768)
(3, 378, 768)
(1, 400, 768)
(1, 258, 768)
(1, 221, 768)
(1, 241, 768)
(1, 462, 768)
(1, 162, 768)
(1, 340, 768)
(1, 223, 768)
(1, 328, 768)
(1, 328, 768)
(1, 143, 768)
(1, 488, 768)
(1, 223, 768)
(1, 80, 768)
(1, 462, 768)
(1, 343, 768)
(1, 277, 768)
(2, 297, 768)
(1, 273, 768)
(1, 364, 768)
(2, 460, 768)
(1, 367, 768)
(2, 382, 768)
(1, 442, 768)
(1, 272, 768)
(2, 410, 768)
(2, 458, 768)
(2, 439, 768)
(1, 353, 768)
(2, 451, 768)
(2, 303, 768)
(1, 264, 768)
(2, 341, 768)
(1, 195, 768)
(2, 366, 768)
(2, 311, 768)
(2, 271, 768)
(3, 374, 768)
(1, 266, 768)
(1, 89, 768)
(2, 386, 768)
(1, 141, 768)
(1, 181, 768)
(1, 348, 768)
(2, 263, 768)
(1, 139, 768)
(1, 243, 768)
(1, 340, 768)
(1, 351, 768)
(1, 428, 768)
(1, 182, 768)
(1, 453, 768)
(2, 427, 768)
(1, 411, 768)
(1, 251, 768)
(2, 418, 768)
(1, 344, 768)
(1, 319, 768)
(2, 346, 768)
(1, 364, 768)
(2, 257, 768)
(2, 411, 768)
(1, 395, 768)
(1, 350, 768)
(3, 392, 768)
(3, 504,

In [3]:
import numpy as np
from os import listdir
from os.path import isfile, join
from pathlib import Path
from Bio import SeqIO


FASTA_FILE_PATH = "dataset/covid19/covid19_proteins.fasta"
lens = []

for count, seqrecord in enumerate(SeqIO.parse(FASTA_FILE_PATH, "fasta")):

    lens.append(len(str(seqrecord.seq)))

print(np.mean(lens))
print(np.max(lens))



885.0909090909091
7096


In [1]:
# export the geneIDs given a xml uniprot file
from Bio import SeqIO

records = list(SeqIO.parse("/storagenfs/m.tolloso/BioEmbedding/dataset/hemoglobin/hemoglobin.xml", "uniprot-xml"))
geneIDs = []

with_gid = 0

xml_names = []
for record in records:
    for i in record.dbxrefs:
        if i.startswith("GeneID:"):
            geneID = i.split(":")[1]
            geneIDs.append(geneID)
            with_gid += 1
            break
    


print(with_gid)

# with open("emoglobina_ids.txt", 'w') as f:
#     for i in geneIDs:
#         f.write(i + "\n")



5332


In [5]:
# given a genbank(full) file, for each proteome select the superset proteins (CDS) and add them to a fasta file

from Bio import SeqIO
import Bio

no_ids = 0

def to_simple_location(f_list):
    for feature in f_list: 
        if type(feature.location) == Bio.SeqFeature.CompoundLocation:
            feature.location = (int(feature.location.parts[0].start) , int(feature.location.parts[1].end))
        else:
            feature.location = (int(feature.location.start), int(feature.location.end))

def remove_subsets(f_list):
    for i in range(len(f_list)):
        for j in range(i+1, len(f_list)):
            if f_list[i].location[0] <= f_list[j].location[0] and f_list[i].location[1] >= f_list[j].location[1]:
                # print(f'removed protein {f_list[j].qualifiers["protein_id"]}')
                f_list.pop(j)
                break
            elif f_list[j].location[0] <= f_list[i].location[0] and f_list[j].location[1] >= f_list[i].location[1]:
                # print(f'removed protein {f_list[i].qualifiers["protein_id"]}')
                f_list.pop(i)
                break
            else:
                continue
        break
    return f_list


gb_records = list(SeqIO.parse('dataset/covid19/covid19.gb','genbank'))
final_proteins = []

for gb_record in gb_records:
    CDS_features = []

    for feature in gb_record.features:
        if feature.type == 'CDS':
            CDS_features.append(feature) 

    to_simple_location(CDS_features)

    # print(len(CDS_features))
    remove_subsets(CDS_features)
    # print(len(CDS_features))

    for feature in CDS_features:
        # print(feature.location, feature.qualifiers['protein_id'], feature.qualifiers['translation'])

        try:
            final_proteins.append((feature.qualifiers['protein_id'], feature.qualifiers['translation']))
        except:
            no_ids +=1


print(len(final_proteins))
print(no_ids)  
 

# output_file = f"dataset/covid19/covid19_proteins.fasta"

# with open(output_file, 'w') as fasta_file:
#     for entry in final_proteins:
#         description, sequence = entry
#         fasta_file.write(f'>{description[0]}\n{sequence[0]}\n')




847
0


In [6]:
from Bio import SeqIO
import numpy as np

gb_records = list(SeqIO.parse('/storagenfs/m.tolloso/BioEmbedding/dataset/meningite/meningite_tr.fasta','fasta'))

lens = []

for gb_record in gb_records:
    lens.append(len(gb_record.seq))

print(np.min(lens))

2203850


In [5]:
from Bio import SeqIO
import os
import numpy as np 

path = '/storagenfs/m.tolloso/BioEmbedding/dataset/covid19/embeddings/alphafold'

# for each file in the folder 'path'

ids = os.listdir(path)


tensor1 = np.load(path + '/' + ids[0])
tensor2 = np.load(path + '/' + ids[3])

print(tensor1.shape)
print(tensor2.shape)

# concat the tensors over the second axis

concat = np.concatenate((tensor1, tensor2), axis=1)

print(concat.shape)

(1, 61, 384)
(1, 222, 384)
(1, 283, 384)


In [4]:
from Bio import SeqIO
import os
import numpy as np 

records = list(SeqIO.parse('dataset/batterio/batterio_tr.fasta','fasta'))

geneIDS = []

for record in records:
    id = record.id
    geneID = record.description.split("[GeneID=")[1].split(']')[0]
    # geneIDS.append(geneID)

    os.rename(f'dataset/batterio/embeddings/dnabert/{id}.npy', f'dataset/batterio/embeddings/dnabert/{geneID}.npy')
    # print(id, geneID)

# print(len(geneIDS))
# print(len(set(geneIDS)))




In [8]:
import re

def extract_gene_id(fasta_header):
    match = re.search(r'\[GeneID=(\d+)\]', fasta_header)
    return match.group(1) if match else None


def process_fasta_file(input_file_path, output_file_path):
    with open(input_file_path, 'r') as input_file, open(output_file_path, 'w') as output_file:
        for line in input_file:
            if line.startswith('>'):

                gene_id = extract_gene_id(line)
                
                # Modify the header to include GeneID
                modified_header = line.replace('>', f'>{gene_id} ')
                output_file.write(modified_header)
                
                # Update current header for the new sequence
                current_header = line.strip()
            else:
                # Continue writing sequence lines
                output_file.write(line)



# Replace 'input.fasta' and 'output.fasta' with the actual file paths
input_file_path = '/storagenfs/m.tolloso/BioEmbedding/dataset/batterio/batterio_tr.fasta'
output_file_path = '/storagenfs/m.tolloso/BioEmbedding/dataset/batterio/batterio_tr_new.fasta'
process_fasta_file(input_file_path, output_file_path)


In [5]:
from Bio import SeqIO
import os

records = list(SeqIO.parse('dataset/mouse/mouse_tr.fasta','fasta'))
files = os.listdir('dataset/mouse/embeddings/dnabert')

ids = [record.id for record in records]
embs = [file.split('.')[0] for file in files]


for id in ids:
    if id not in embs:
        print(id)  





In [28]:
# aggegate the satb fasta files
from Bio import SeqIO
from pathlib import Path


files = ['Satb1_Orthologues_protein_sequences.txt', 'Satb1_Satb2_protein_sequences.txt', 'Satb2_Orthologues_protein_sequences.txt']
files_tr = ['Satb1_Orthologues_tr_sequences.txt', 'Satb1_Satb2_tr_sequences.txt', 'Satb2_Orthologues_tr_sequences.txt']

transcript_dict = {}

for file_tr in files_tr:
    records_fasta = list(SeqIO.parse("/storagenfs/m.tolloso/BioEmbedding/dataset/bacterium/bacterium_proteins.faa", "fasta"))

    for record_tr in records_fasta:
        transcript_dict[record_tr.id] = record_tr.description.split('[GeneID=')[1][:-1]



embedder = 'alphafold'



files = os.listdir(f'/storagenfs/m.tolloso/BioEmbedding/dataset/bacterium/embeddings/{embedder}')
for file in files :

    # print(f'/storagenfs/m.tolloso/BioEmbedding/dataset/bacterium/embeddings/{embedder}/{protein_name}.npy')
    # print(f'/storagenfs/m.tolloso/BioEmbedding/dataset/bacterium/embeddings/{embedder}/{transcript_dict[protein_name]}.npy')


    os.rename(f'/storagenfs/m.tolloso/BioEmbedding/dataset/bacterium/embeddings/{embedder}/{file}', 
                f'/storagenfs/m.tolloso/BioEmbedding/dataset/bacterium/embeddings/{embedder}/{file}'[:-8])

          

transcript_dict

# with open('dataset/satb2/satb2_list.txt', 'w') as of:

#     for file in files:
#         records = list(SeqIO.parse(f'dataset/satb2/{file}', 'fasta'))
        
#         for record in records:
            
#             if len(record.seq) == 0:
#                 of.write(f'>gene:{record.id}')
#                 seq = transcript_dict[record.id]
#             else:
#                 of.write(f' protein:{record.id}\n')
#                 of.write(f'{seq}\n')
                 

{'WP_002215295.1': '61282335',
 'WP_002260475.1': '61282584',
 'WP_002229878.1': '61280682',
 'WP_002229851.1': '61280727',
 'WP_002216981.1': '61281572',
 'WP_002215290.1': '61282337',
 'WP_002229179.1': '61280887',
 'WP_002221507.1': '61280660',
 'WP_002229995.1': '61282601',
 'WP_002229331.1': '61281137',
 'WP_002212455.1': '61282011',
 'WP_002215455.1': '61282244',
 'WP_002226892.1': '61282336',
 'WP_002215287.1': '61282338',
 'WP_002226887.1': '61282360',
 'WP_002229965.1': '61282648',
 'WP_002229536.1': '61281483',
 'WP_002229840.1': '61281884',
 'WP_002213013.1': '61281624',
 'WP_002214623.1': '61280649',
 'WP_002215466.1': '61282238',
 'WP_002216405.1': '61282060',
 'WP_002230034.1': '61282534',
 'WP_002261325.1': '61280666',
 'WP_002260404.1': '61280926',
 'WP_002213936.1': '61281052',
 'WP_002229373.1': '61281216',
 'WP_002216904.1': '61281638',
 'WP_002216735.1': '61281801',
 'WP_002259859.1': '61281835',
 'WP_002229835.1': '61281871',
 'WP_002212592.1': '61281897',
 'WP_002

In [8]:
from Bio import SeqIO

file_name = f'dataset/satb2/satb2_tr.txt'

import FastaValidator

return_code = FastaValidator.fasta_validator(file_name)
print(return_code)



records_tr = list(SeqIO.parse(f'dataset/satb2/satb2_tr.txt', 'fasta'))


ids = []

for record in records_tr:
    print(record.id)



0
ENSNLET00000001146
ENSNLET00000057739
ENSNLET00000051756
ENSNLET00000056267
ENSGGOT00000023995
ENSGGOT00000001792
ENSGGOT00000064749
ENSGGOT00000057507
ENSGGOT00000043887
ENSOANT00000049677
ENSOANT00000007917
ENSOANT00000059119
ENSPTRT00000084390
ENSPTRT00000027458
ENSPTRT00000094143
ENSPTRT00000103679
ENSPPYT00000041113
ENSPPYT00000016369
ENSPPYT00000058026
ENSPPAT00000027407
ENSPPAT00000027411
ENSPPAT00000027415
ENSPPAT00000027417
ENSSHAT00000018250
ENSSHAT00000018251
ENSSHAT00000041558
ENSCHOT00000002151
ENSMEUT00000013134
ENSDNOT00000023597
ENSEEUT00000010438
ENSETET00000012405
ENSCJAT00000007022
ENSCJAT00000076471
ENSCJAT00000132627
ENSCATT00000059848
ENSCATT00000059856
ENSCATT00000059858
ENSCATT00000059862
ENSCATT00000059864
ENSCATT00000059865
ENSMMUT00000095043
ENSMMUT00000026713
ENSMMUT00000088916
ENSMMUT00000106645
ENSPANT00000064150
ENSPANT00000042572
ENSPANT00000024101
ENSPANT00000047909
ENSMNET00000044292
ENSMNET00000044297
ENSMNET00000044302
ENSMNET00000044309
ENSMLET000

In [32]:
# export the geneIDs given a xml uniprot file
from Bio import SeqIO
from autoembedding.utils import get_gene_id
import os

# records_xml = list(SeqIO.parse("/storagenfs/m.tolloso/BioEmbedding/dataset/bacterium/Neisseria_meningitidis.xml", "uniprot-xml"))
records_fasta = list(SeqIO.parse("/storagenfs/m.tolloso/BioEmbedding/dataset/bacterium/bacterium_tr_new.fasta", "fasta"))



complete = 0

# for record in records_fasta:
#     fasta_IDs.append(record.id)

for record in records_fasta:


    os.path.exists(f"/storagenfs/m.tolloso/BioEmbedding/dataset/bacterium/embeddings/alphafold/{record.id}.npy")
    complete += 1


print(complete)


903


In [8]:
final = list(set(xml_IDs).intersection(set(fasta_IDs)))
len(final)

903

In [6]:
from Bio import SeqIO
with open('dataset/bacterium/bacterium_tr_new.fasta', 'w') as f:

    for record in records_fasta:
        if record.id in final:
            # write the record to the output file
            SeqIO.write(record, f, "fasta")

In [1]:
from Bio import SeqIO
from itertools import combinations


command = './clustalo-1.2.4-Ubuntu-x86_64 --infile tmp/test.fasta --threads 40 --MAC-RAM 8000 -v --verbose --guidetree-out tmp/gt.dnd --distmat-out tmp/dm.matrix --outfmt clustal --resno --outfile tmp/of.clustal_num --output-order tree-order --seqtype dna'
command_nodm = './clustalo-1.2.4-Ubuntu-x86_64 --infile tmp/test.fasta --threads 40 -v --verbose --auto --outfile tmp/of.clustal_num  --seqtype dna'


mengite_tr = list(SeqIO.parse("/storagenfs/m.tolloso/BioEmbedding/dataset/meningite/meningite_tr.fasta", "fasta"))



for record1, record2 in combinations(mengite_tr, 2):
    with open("test.fasta", "w") as f:
        SeqIO.write([record1, record2], f, "fasta")
        break


In [10]:

import pickle
from scipy.cluster import hierarchy

res = pickle.load(open("tmp/res_covid19.pkl", "rb"))
Z = res['embeddings_linkage_matrix']
leaf_names = res['embeddings_IDs']


def get_newick(node, parent_dist, leaf_names, newick='') -> str:
    """
    Convert sciply.cluster.hierarchy.to_tree()-output to Newick format.

    :param node: output of sciply.cluster.hierarchy.to_tree()
    :param parent_dist: output of sciply.cluster.hierarchy.to_tree().dist
    :param leaf_names: list of leaf names
    :param newick: leave empty, this variable is used in recursion.
    :returns: tree in Newick format
    """
    if node.is_leaf():
        return "%s:%.2f%s" % (leaf_names[node.id], parent_dist - node.dist, newick)
    else:
        if len(newick) > 0:
            newick = "):%.2f%s" % (parent_dist - node.dist, newick)
        else:
            newick = ");"
        newick = get_newick(node.get_left(), node.dist, leaf_names, newick=newick)
        newick = get_newick(node.get_right(), node.dist, leaf_names, newick=",%s" % (newick))
        newick = "(%s" % (newick)
        return newick

tree = hierarchy.to_tree(Z, False)
newick = get_newick(tree, tree.dist, leaf_names)
with open("results_trees/covid19_esm.nwk", "w") as f:
    f.write(newick)